## Setup

In [106]:
from xml.etree import ElementTree as ET
import pandas as pd
from tqdm.auto import tqdm
import requests
from datetime import datetime

pd.set_option("display.max_colwidth", 0)
pd.set_option("display.max_columns", None)

df = pd.read_csv("../Datasets/newspapers_ger_1914_part_1_only_ads.csv")
print(df.columns)
print(df.shape[0])


Index(['page_id', 'pagenumber', 'paper_title', 'provider_ddb_id', 'provider',
       'zdb_id', 'publication_date', 'place_of_distribution', 'language',
       'thumbnail', 'pagefulltext', 'pagename', 'preview_reference',
       'plainpagefulltext', 'paper_id', 'file_path', 'classification',
       'probability', 'true_ad'],
      dtype='object')
101


In [138]:
ALTO_URL = "preview_reference"

In [3]:
df[['page_id', 'pagenumber', 'paper_title', 'provider_ddb_id', 'provider',
       'zdb_id', 'publication_date', 'place_of_distribution', 'language',
       'thumbnail', 'pagefulltext', 'pagename', 'preview_reference',
       'paper_id', 'file_path', 'classification',
       'probability', 'true_ad']].head(2)

,page_id,pagenumber,paper_title,provider_ddb_id,provider,zdb_id,publication_date,place_of_distribution,language,thumbnail,pagefulltext,pagename,preview_reference,paper_id,file_path,classification,probability,true_ad
0,22KPDIU2CCI3K3INHQ2TVFLRDDZCICYM-FILE_0004_DDB_FULLTEXT,4,Dresdner Journal : Königlich Sächsischer Staatsanzeiger ; Verordnungsblatt der Ministerien und der Ober- und Mittelbehörden,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Universitätsbibliothek Dresden,2889116-8,1914-03-28 12:00:00,['Dresden'],['ger'],268a11f3-8b28-49b0-a081-e6b7396f8b84,['/data/altos/22/KP/22KPDIU2CCI3K3INHQ2TVFLRDDZCICYM-FILE_0004_DDB_FULLTEXT'],FILE_0004_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/binary/b5922e20-c39b-4464-8f92-56727cd452ba.xml,22KPDIU2CCI3K3INHQ2TVFLRDDZCICYM,N:\Jobads\hermes_images\22KPDIU2CCI3K3INHQ2TVFLRDDZCICYM\22KPDIU2CCI3K3INHQ2TVFLRDDZCICYM_3.jpeg,1.0,0.944500,1
1,22QRMSPWH2VFNZT5DNCCRGMBF4GQOMPE-FILE_0014_DDB_FULLTEXT,14,Süddeutsche Zeitung : für deutsche Politik und Volkswirtschaft,VNHXUCEEKHOUSYH4NVOUBHJGSRMOGK7J,Württembergische Landesbibliothek,3135995-4,1914-01-14 12:00:00,['Stuttgart'],['ger'],af2bbc1f-ecb0-40ae-aad9-7e2af357b415,['/data/altos/22/QR/22QRMSPWH2VFNZT5DNCCRGMBF4GQOMPE-FILE_0014_DDB_FULLTEXT'],FILE_0014_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/binary/b41d58fe-cb64-4a5b-b692-104dd8a898a8.xml,22QRMSPWH2VFNZT5DNCCRGMBF4GQOMPE,N:\Jobads\hermes_images\22QRMSPWH2VFNZT5DNCCRGMBF4GQOMPE\22QRMSPWH2VFNZT5DNCCRGMBF4GQOMPE_13.jpeg,1.0,0.886392,1


In [145]:

def extract_regions_from_alto_xml(root):
    # ET.dump(root)
    regions = []
    ns = {'alto': root.tag.split('}')[0].strip('{')}
    textblocks = root.findall(".//alto:Layout/alto:Page/alto:PrintSpace//alto:TextBlock", ns)
    for bi, block in enumerate(textblocks):
        block_text = []
        # print(block.attrib)
        for li, line in enumerate(block):
            line_data = []
            line_text = []
            for wi, word in enumerate(line):
                if word.tag.endswith("String"):
                    line_data.append(word.attrib)
                    if word.attrib.get("CONTENT"):
                        line_text.append(word.attrib.get("CONTENT"))
            if line_text:
                block_text.append(" ".join(line_text))
        regions.append({**block.attrib, "text": "\n".join(block_text)})
        # print(tb)
        # for i, element in enumerate(tb):
        #     print(f"\t{i+1}. {element}")
        #     print(f"\t\t{element.attrib}")
    return regions


In [ ]:


ERROR_REPORT = "../Datasets/regions_error_report.md"

with open(ERROR_REPORT, "w") as f:
    f.write(f"\n# {datetime.now()}\n")
    region_data = []
    for i, row in tqdm(df.iterrows(), total=df.shape[0]):
        row_data = row.to_dict()
        row_data.pop("plainpagefulltext")
        try:
            response = requests.get(row[ALTO_URL].strip())
            response.raise_for_status()
            root = ET.fromstring(response.content)
            regions = extract_regions_from_alto_xml(root)
            if not regions:
                f.write(f"- {row.page_id}: NO REGIONS FOUND.\n")
            else:
                region_data.extend([{**row_data, **r} for r in regions])

        except Exception as e:
            region_data.append({**row_data})
            f.write(f"- {row.page_id}: {str(e)}\n")

In [146]:
rdf = pd.DataFrame(region_data)

In [160]:
rdf.drop(["STYLEREFS"], inplace=True, axis=1)

In [161]:
rdf.sample(10)

,page_id,pagenumber,paper_title,provider_ddb_id,provider,zdb_id,publication_date,place_of_distribution,language,thumbnail,pagefulltext,pagename,preview_reference,paper_id,file_path,classification,probability,true_ad,ID,HEIGHT,WIDTH,VPOS,HPOS,text
3349,2GRIAGZDDFBXOZW5EXM7E4VCYSD6QXSH-FILE_0034_DDB_FULLTEXT,34,Hamburger Fremdenblatt,BZVTR553HLJBDMQD5NCJ6YKP3HMBQRF4,Staats- und Universitätsbibliothek Hamburg Carl von Ossietzky,3024925-9,1914-01-08 12:00:00,['Hamburg'],de,25b6c7a8-bbc0-4907-b807-1e447744bdbe,['/data/altos/2G/RI/2GRIAGZDDFBXOZW5EXM7E4VCYSD6QXSH-FILE_0034_DDB_FULLTEXT'],FILE_0034_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/binary/d633a39a-cb63-424b-97e0-446193843935.xml,2GRIAGZDDFBXOZW5EXM7E4VCYSD6QXSH,N:\Jobads\hermes_images\2\2GRIAGZDDFBXOZW5EXM7E4VCYSD6QXSH_33.jpeg,1.0,0.933662,1,Page1_Block175,388,526,4230,508,"Äuto-Oele u. -Fette.\nLrtte Firma der Brnndje sucht\n*ct halb acbilbetc Tarne, bit serlig\nItenoyrfll'bicrt unb Sdircibmafdrint\nschreibt und auch in der Lage ift,\nseiche flnml^rrnbriii\n»der nach Angabe in «lebigen,\nkamen, die in der\nBrauche bereit» versiert Rnb. er\nhalten beu Rkirinift. irdach nicht\nBebinawng. Cffert nut. ®. 32S\nan bip ftvit d«ch tsfca r.- ,'"
2387,2CPGV2KFS7JN2XXA5FL3LQPNR5RWZN5C-FILE_0010_DDB_FULLTEXT,10,Dresdner neueste Nachrichten,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Universitätsbibliothek Dresden,2904126-0,1914-03-11 12:00:00,['Dresden'],de,59d6b3d8-5cff-42e8-bf38-f52b73e684bb,['/data/altos/2C/PG/2CPGV2KFS7JN2XXA5FL3LQPNR5RWZN5C-FILE_0010_DDB_FULLTEXT'],FILE_0010_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/binary/9e07cb4c-ada0-4dbb-a6db-9b3c7a20fffc.xml,2CPGV2KFS7JN2XXA5FL3LQPNR5RWZN5C,N:\Jobads\hermes_images\2\2CPGV2KFS7JN2XXA5FL3LQPNR5RWZN5C_9.jpeg,1.0,0.935390,1,P10_TB00168,155,374,2796,1999,Für lot-ihn geweckt- Knaben\n(R-ca«chüler«s wird otute und\nnicht teure Pension in Nähe d.\nVivtbumftraßc gesucht. Gef-\nLlngek unt. »Schülerucnsion«\npoftlaaemd Cossebaude. sce2B23\nW
2150,2CPGV2KFS7JN2XXA5FL3LQPNR5RWZN5C-FILE_0009_DDB_FULLTEXT,9,Dresdner neueste Nachrichten,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Universitätsbibliothek Dresden,2904126-0,1914-03-11 12:00:00,['Dresden'],de,59d6b3d8-5cff-42e8-bf38-f52b73e684bb,['/data/altos/2C/PG/2CPGV2KFS7JN2XXA5FL3LQPNR5RWZN5C-FILE_0009_DDB_FULLTEXT'],FILE_0009_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/binary/6a0adfd4-2b52-45fb-a450-8241fe02c37b.xml,2CPGV2KFS7JN2XXA5FL3LQPNR5RWZN5C,N:\Jobads\hermes_images\2\2CPGV2KFS7JN2XXA5FL3LQPNR5RWZN5C_8.jpeg,1.0,0.939776,1,P9_TB00152,176,374,4500,921,"Ostermäszem wetz. die Sthile\nverl» facht leichte Stelle zu ! od. 2\nKind. NäthispipLQ 1. k. 111164\nKIND VKFIIIIZIUU w. zu\ngut-sc schon Xätig wet, sucht Is. c- q.\ntell. in Pup. D. Titteb Dresd-\nNeuft.. Sehr-mir Str. As, UUSLO"
3164,2GRIAGZDDFBXOZW5EXM7E4VCYSD6QXSH-FILE_0033_DDB_FULLTEXT,33,Hamburger Fremdenblatt,BZVTR553HLJBDMQD5NCJ6YKP3HMBQRF4,Staats- und Universitätsbibliothek Hamburg Carl von Ossietzky,3024925-9,1914-01-08 12:00:00,['Hamburg'],de,25b6c7a8-bbc0-4907-b807-1e447744bdbe,['/data/altos/2G/RI/2GRIAGZDDFBXOZW5EXM7E4VCYSD6QXSH-FILE_0033_DDB_FULLTEXT'],FILE_0033_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/binary/e427c792-b6bd-45f9-bfba-3b8d0d9a749c.xml,2GRIAGZDDFBXOZW5EXM7E4VCYSD6QXSH,N:\Jobads\hermes_images\2\2GRIAGZDDFBXOZW5EXM7E4VCYSD6QXSH_32.jpeg,1.0,0.942062,1,Page1_Block109,768,528,1186,3955,"Vcbrliitrt\nfür LdeditioiiS und Cdiiffabrtf#\ngeschalt jurn 1. April gesucht. Der»\nfiütuna nwnatlich 15. 25, 30 Cff.\nI. 3.380 an bte ßrV. Lrbg. »rbbtj\n<SLür anetfebene Assckurantfirtn#\nwirb in Citern 1914 ein lieh»\nlina. Sohn achtbarer Eltern. gege#\nhohe Dorgütung gesucht. Cff. unt\nB 3.392 an bte Ort», ibg ffrbbU*\ntcbvlina\nfür die Hamburger Filiale eines\nLeidenfabrik gegen übliche Dee»\ngütung gesucht. (gute Sehnt»\nfenntniffe erforderlich Schriftlich«\nCif, u. T. 3 2- i-rp. Hbg -rrbbl^\nWir suchen mm 1. A

In [162]:
rdf.to_csv("../Datasets/newspapers_ger_1914_part_1_only_ads_REGIONS.csv", index=False)

In [163]:
rdf.columns

Index(['page_id', 'pagenumber', 'paper_title', 'provider_ddb_id', 'provider',
       'zdb_id', 'publication_date', 'place_of_distribution', 'language',
       'thumbnail', 'pagefulltext', 'pagename', 'preview_reference',
       'paper_id', 'file_path', 'classification', 'probability', 'true_ad',
       'ID', 'HEIGHT', 'WIDTH', 'VPOS', 'HPOS', 'text'],
      dtype='object')

In [164]:
rdf.shape[0]

3395

In [166]:
assert len(rdf.page_id.unique()) == df.shape[0]